## CS-GY 6953 / ECE-GY 7123     Fall 2021
# Project code - Different MLP Combinations

### Project Title: Comparison of Machine Learning Methods and Discrete Choice Models in Predicting the Travel Mode Choices of NYC Residents

## Bingqing Liu
### PhD Candidate in Transportation Engineering and Planning
### bl2453@nyu.edu

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

## Project purpose
### Using the 2019 NYC Citywide Mobility Survey data to predict travel mode choice using deep learning models
### Data: 
### - trip data: mode of the trip ( y ), other features of the trip
### - traveler demographic data: features of the traveler of the trip, merged to the trip data 

-------------

# Trip data
### Main dataset

In [2]:
trips = pd.read_csv("Citywide_Mobility_Survey_-_Trip_Survey_2019.csv")

----------

# Person data
### demographic and behavioral information of the traveler
### merged to the trip dataset to add more x

In [3]:
person_id = np.array(trips['person_id'])

In [4]:
unique_person = np.unique(person_id)

In [5]:
len(unique_person)

3037

In [6]:
persons = pd.read_csv("Citywide_Mobility_Survey_-_Person_Survey_2019.csv")
persons.shape

(8286, 165)

In [7]:
trips_person = trips.merge(persons, left_on = "person_id", right_on = "person_id")

--------

# Dropping useless features

In [8]:
### check the original columns
col = trips_person.columns
for i in range(len(col)):
    print(i, col[i])

0 hh_id_x
1 trip_weight
2 home_cms_zone
3 survey_mode_x
4 person_id
5 person_num_x
6 day_num
7 travel_date
8 travel_date_dow
9 trip_id
10 trip_num
11 leg_num
12 unlinked_trip
13 linked_trip_num
14 survey_complete
15 depart_time
16 arrive_time
17 o_cms_zone
18 d_cms_zone
19 trip_duration
20 trip_distance
21 o_purpose
22 o_purpose_category
23 d_purpose
24 d_purpose_category
25 num_non_hh_travelers
26 num_hh_travelers
27 num_travelers
28 hh_member_1
29 hh_member_2
30 hh_member_3
31 hh_member_4
32 hh_member_5
33 hh_member_6
34 hh_member_7
35 hh_member_8
36 hh_member_9
37 hh_member_10
38 hh_member_11
39 mode_type
40 mode_type_detailed
41 sustainable_mode
42 mode_1
43 mode_2
44 mode_3
45 mode_4
46 driver
47 park_location
48 park_pay
49 park_cost
50 used_transit
51 transit_access
52 transit_egress
53 taxi_cost
54 taxi_cost_amount
55 bike_park_location
56 scooter_park_location
57 analyst_split
58 analyst_merge
59 analyst_split_loop
60 user_merged
61 user_split
62 added_trip
63 high_speed_flag


In [9]:
trips_person = trips_person.drop(["hh_id_y",
                                  "survey_mode_y",
                                  "person_num_y",
                                  "survey_mode_x",
                                  "travel_date",
                                  "travel_date_dow",
                                  "trip_num",
                                  'trip_weight',
                                  "leg_num",
                                  "linked_trip_num",
                                  "depart_time",
                                  "arrive_time",
                                  "mode_1",
                                  "mode_2",
                                  "mode_3",
                                  "mode_4",
                                  "analyst_split",
                                  "analyst_merge",
                                  "analyst_split_loop",
                                  "user_merged",
                                  'user_split',
                                  'added_trip',
                                  'invalid_purpose_flag',
                                  'o_purpose',
                                  'o_purpose_category',
                                  'd_purpose',
                                  'd_purpose_category',
                                  'weight',
                                  "is_participant",
                                  'num_days',
                                  'num_days_complete',
                                  'first_travel_date',
                                  'last_travel_date',
                                  'made_trips',
                                  'relationship',
                                  'missing_work_location',
                                  'missing_school_location',
                                  'school_mode_scooter',
                                  'school_scooter_park',
                                  "hh_member_1",
                                  "hh_member_2",
                                  "hh_member_3",
                                  "hh_member_4",
                                  "hh_member_5",
                                  "hh_member_6",
                                  "hh_member_7",
                                  "hh_member_8",
                                  "hh_member_9",
                                  "hh_member_10",
                                  "hh_member_11",
                                  "person_id",
                                  "person_num_x",
                                  "day_num",
                                  "hh_id_x",
                                  "survey_complete"], axis=1)
len(trips_person.columns)

186

In [10]:
### check the final columns
col = trips_person.columns
for i in range(len(col)):
    print(i, col[i])

0 home_cms_zone
1 trip_id
2 unlinked_trip
3 o_cms_zone
4 d_cms_zone
5 trip_duration
6 trip_distance
7 num_non_hh_travelers
8 num_hh_travelers
9 num_travelers
10 mode_type
11 mode_type_detailed
12 sustainable_mode
13 driver
14 park_location
15 park_pay
16 park_cost
17 used_transit
18 transit_access
19 transit_egress
20 taxi_cost
21 taxi_cost_amount
22 bike_park_location
23 scooter_park_location
24 high_speed_flag
25 long_distance_flag
26 o_purpose_imputed
27 d_purpose_imputed
28 o_purpose_category_imputed
29 d_purpose_category_imputed
30 d_distance_home
31 d_distance_work
32 d_distance_school
33 o_location_type
34 d_location_type
35 o_congestion
36 d_congestion
37 cms_zone
38 num_trips
39 num_walk_trips
40 num_transit_trips
41 num_bike_trips
42 num_taxi_trips
43 num_tnc_trips
44 num_complete_weekend_days
45 num_complete_weekdays
46 smartphone_type
47 age
48 employment
49 worker
50 student
51 license
52 vehicle
53 congestion
54 drive_cbd_freq
55 planning_apps
56 job_type
57 jobs_count
58

### unique ID for each row: "trip_id"
### number of columns after dropping : 186
### number of x after dropping: 186 - 2 - 1 = 183 (exclude "trip_id" and y variables "mode_type" and "mode_type_detailed")


---------------

# y: Travel mode
### "mode_type" or "mode_type_detailed" (can try the 2 of them seperately, and see which one is better)

In [11]:
trips_person['mode_type'].describe()

count    85459.000000
mean        80.570683
std        258.839050
min          1.000000
25%          6.000000
50%          7.000000
75%          9.000000
max        995.000000
Name: mode_type, dtype: float64

In [12]:
trips_person['mode_type_detailed'].describe()

count    85459.000000
mean       102.803953
std        252.985668
min          1.000000
25%         16.000000
50%         32.000000
75%         48.000000
max        995.000000
Name: mode_type_detailed, dtype: float64

In [13]:
### unique values of 'mode_type'
modes = np.array(trips_person['mode_type'])
mode_unique, mode_counts = np.unique(modes, return_counts=True)
for u,c in zip(mode_unique, mode_counts):
    print("mode type: %d and its count: %d"%(u,c))

mode type: 1 and its count: 696
mode type: 2 and its count: 1244
mode type: 3 and its count: 662
mode type: 4 and its count: 5355
mode type: 5 and its count: 441
mode type: 6 and its count: 13263
mode type: 7 and its count: 23678
mode type: 8 and its count: 1342
mode type: 9 and its count: 31060
mode type: 10 and its count: 1379
mode type: 995 and its count: 6339


In [14]:
### unique values of 'mode_type_detailed'
detailed_modes = np.array(trips_person['mode_type_detailed'])
detailed_modes_unique, detailed_modes_counts = np.unique(detailed_modes, return_counts=True)
for u,c in zip(detailed_modes_unique, detailed_modes_counts):
    print("mode type: %d and its count: %d"%(u,c))
    

mode type: 1 and its count: 321
mode type: 2 and its count: 53
mode type: 3 and its count: 1066
mode type: 4 and its count: 178
mode type: 5 and its count: 298
mode type: 6 and its count: 662
mode type: 7 and its count: 849
mode type: 8 and its count: 288
mode type: 9 and its count: 101
mode type: 10 and its count: 52
mode type: 11 and its count: 13205
mode type: 12 and its count: 58
mode type: 13 and its count: 87
mode type: 14 and its count: 99
mode type: 15 and its count: 429
mode type: 16 and its count: 3777
mode type: 17 and its count: 7
mode type: 18 and its count: 24
mode type: 19 and its count: 80
mode type: 20 and its count: 114
mode type: 21 and its count: 55
mode type: 22 and its count: 75
mode type: 23 and its count: 24
mode type: 24 and its count: 24
mode type: 25 and its count: 15750
mode type: 26 and its count: 2859
mode type: 27 and its count: 279
mode type: 28 and its count: 2
mode type: 29 and its count: 8
mode type: 30 and its count: 1141
mode type: 31 and its count:

In [15]:
trips_person

,home_cms_zone,trip_id,unlinked_trip,o_cms_zone,d_cms_zone,trip_duration,trip_distance,num_non_hh_travelers,num_hh_travelers,num_travelers,...,race_native_hawaiian,race_white,race_other,disability_hearing,disability_seeing,disability_walking,disability_mobility,disability_dressing,disability_concentrating,disability_none
0,Northern Bronx,1900023501001,1,Manhattan Core,Southern Bronx,11.1,13.4,0,1,1,...,0,0,0,0,0,0,0,0,0,1
1,Northern Bronx,1900023501002,0,Southern Bronx,Northern Bronx,7.0,3.6,-9998,-9998,-9998,...,0,0,0,0,0,0,0,0,0,1
2,Northern Bronx,1900023501003,0,Northern Bronx,Southern Bronx,16.7,3.4,-9998,-9998,-9998,...,0,0,0,0,0,0,0,0,0,1
3,Northern Bronx,1900023501004,0,Southern Bronx,Northern Bronx,9.1,3.7,0,1,1,...,0,0,0,0,0,0,0,0,0,1
4,Northern Bronx,1900023501005,0,Northern Bronx,Northern Bronx,4.4,1.0,-9998,-9998,-9998,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85454,Inner Brooklyn,19998918901062,0,Manhattan Core,Manhattan Core,1.3,0.2,2,2,4,...,0,1,0,0,0,0,0,0,0,1
85455,Inner Brooklyn,19998918901063,0,Manhattan Core,Manhattan Core,15.6,0.9,2,2,4,...,0,1,0,0,0,0,0,0,0,1
85456,Inner Brooklyn,19998918901064,0,Manhattan Core,Manhattan Core,13.4,0.8,2,2,4,...,0,1,0,0,0,0,0,0,0,1
85457,Inner Brooklyn,19998918901065,0,Manhattan Core,Manhattan Core,27.3,1.1,2,2,4,...,0,1,0,0,0,0,0,0,0,1


### Prepare X and Y data 

In [16]:
data_Y = trips_person['mode_type']
data_X = trips_person.drop(columns=['mode_type', 'mode_type_detailed', 'trip_id'])
data_X = data_X.fillna(0)
print(data_X.shape, data_Y.shape)


(85459, 183) (85459,)


In [17]:
cat_np = []
num_np = []
X = data_X
Y = data_Y

for col,dtype in zip(X.columns, X.dtypes):
    if dtype == 'O':
        if len(cat_np) == 0:
            col_data = X[col].to_numpy()
            col_data = np.array(['N.A.' if type(i) == int or type(i) == float else i for i in col_data])
            cat_np = col_data
        else:
            col_data = X[col].to_numpy()
            col_data = np.array(['N.A.' if type(i) == int or type(i) == float else i for i in col_data])
            cat_np = np.vstack((cat_np, col_data))
    else:
        num_np.append(list(X[col]))
    
            

In [18]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
if len(cat_np) != 0:
    if len(cat_np.shape) == 1:
        cat_encode = enc.fit_transform(cat_np.reshape(cat_np.shape[0],1))
    else:
        cat_encode = enc.fit_transform(cat_np.T)

    if len(num_np) == 0:
        prepared_X = cat_encode
    else:
        prepared_X = np.hstack((np.array(num_np).T, cat_encode))
else:
    prepared_X = np.array(num_np).T

# y_before_unique, y_before_count = np.unique(Y, return_counts=True)
# print(y_before_unique, y_before_count)
# Y = Y.replace(995, 11)
# y_after_unique, y_after_count = np.unique(Y, return_counts=True)
# print(y_after_unique, y_after_count)
Y = enc.fit_transform(np.array(Y).reshape(Y.shape[0],1))
    
if Y.dtype == 'O':
    prepared_Y = enc.fit_transform(Y.to_numpy().reshape(Y.shape[0],1))
else:
    prepared_Y = Y

In [19]:
print(prepared_X.shape, prepared_Y.shape)


(85459, 183) (85459, 1)


In [20]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [21]:
x_train, x_test, y_train, y_test = train_test_split(prepared_X, 
                                                    prepared_Y, 
                                                    random_state=19,
                                                    test_size=0.2)

In [22]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(68367, 183) (17092, 183) (68367, 1) (17092, 1)


## Experiment with 64 combinations:

- Each layer has choice of hidden units from (64, 128, 256, 512)
- 3 layers


In [49]:
historys = {}
evaluations = {}

hidden_units_candidates = [64, 128, 256, 512]

epochs = 200

out_batch = NBatchLogger(display=20)

for hid1 in hidden_units_candidates:
    for hid2 in hidden_units_candidates:
        for hid3 in hidden_units_candidates:
            hidden_units = [hid1, hid2, hid3]
            key = "_".join([str(i) for i in hidden_units])
            print(key)
            
            model = keras.Sequential([
            #     layers.Dropout(0.2),
                layers.Dense(hidden_units[0], 
                             activation='tanh',
            #                  kernel_regularizer=tf.keras.regularizers.l2(),
                             name="layer1"
                            ),
            #     layers.LeakyReLU(alpha=0.1),
                layers.Dropout(0.5),
                layers.Dense(hidden_units[1], 
                             activation='tanh',
            #                  kernel_regularizer=tf.keras.regularizers.l2(),
                             name="layer2"
                            ),
            #     layers.LeakyReLU(alpha=0.1),
                layers.Dropout(0.5),
                layers.Dense(hidden_units[2],
                             activation='tanh',
            #                  kernel_regularizer=tf.keras.regularizers.l2(),
                             name="layer3"
                            ),
            #     layers.LeakyReLU(alpha=0.1),
                layers.Dropout(0.5),
                layers.Dense(len(np.unique(Y)), activation='softmax')
            ])

            model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

            history = model.fit(x_train.astype('float32'), y_train, 
                                batch_size=128, epochs=epochs, 
                                validation_split=0.2, verbose=1)
            historys[key] = history.history
            evaluation = model.evaluate(x_test, y_test)
            evaluations[key] = evaluation


64_64_64
Epoch 1/200
428/428 [==============================] - 1s 2ms/step - loss: 1.7205 - accuracy: 0.3798 - val_loss: 1.4098 - val_accuracy: 0.4724
Epoch 2/200
428/428 [==============================] - 1s 1ms/step - loss: 1.5073 - accuracy: 0.4456 - val_loss: 1.3663 - val_accuracy: 0.4817
Epoch 3/200
428/428 [==============================] - 1s 1ms/step - loss: 1.4542 - accuracy: 0.4696 - val_loss: 1.2921 - val_accuracy: 0.5290
Epoch 4/200
428/428 [==============================] - 1s 1ms/step - loss: 1.4275 - accuracy: 0.4811 - val_loss: 1.2521 - val_accuracy: 0.5764
Epoch 5/200
428/428 [==============================] - 1s 1ms/step - loss: 1.3901 - accuracy: 0.5092 - val_loss: 1.1984 - val_accuracy: 0.5725
Epoch 6/200
428/428 [==============================] - 1s 1ms/step - loss: 1.3442 - accuracy: 0.5263 - val_loss: 1.1580 - val_accuracy: 0.6060
Epoch 7/200
428/428 [==============================] - 1s 1ms/step - loss: 1.3474 - accuracy: 0.5277 - val_loss: 1.2543 - val_accurac

Epoch 58/200
428/428 [==============================] - 1s 1ms/step - loss: 0.8331 - accuracy: 0.7447 - val_loss: 0.6713 - val_accuracy: 0.7991
Epoch 59/200
428/428 [==============================] - 1s 1ms/step - loss: 0.8464 - accuracy: 0.7361 - val_loss: 0.6221 - val_accuracy: 0.8133
Epoch 60/200
428/428 [==============================] - 1s 1ms/step - loss: 0.8345 - accuracy: 0.7418 - val_loss: 0.5989 - val_accuracy: 0.8354
Epoch 61/200
428/428 [==============================] - 1s 1ms/step - loss: 0.8018 - accuracy: 0.7583 - val_loss: 0.5930 - val_accuracy: 0.8348
Epoch 62/200
428/428 [==============================] - 1s 1ms/step - loss: 0.8255 - accuracy: 0.7494 - val_loss: 0.5962 - val_accuracy: 0.8308
Epoch 63/200
428/428 [==============================] - 1s 1ms/step - loss: 0.7877 - accuracy: 0.7632 - val_loss: 0.6123 - val_accuracy: 0.8165
Epoch 64/200
428/428 [==============================] - 1s 1ms/step - loss: 0.8265 - accuracy: 0.7477 - val_loss: 0.6481 - val_accuracy:

Epoch 115/200
428/428 [==============================] - 1s 1ms/step - loss: 0.7297 - accuracy: 0.7835 - val_loss: 0.5819 - val_accuracy: 0.8413
Epoch 116/200
428/428 [==============================] - 1s 1ms/step - loss: 0.7224 - accuracy: 0.7858 - val_loss: 0.5801 - val_accuracy: 0.8401
Epoch 117/200
428/428 [==============================] - 1s 1ms/step - loss: 0.7756 - accuracy: 0.7676 - val_loss: 0.6276 - val_accuracy: 0.8143
Epoch 118/200
428/428 [==============================] - 1s 1ms/step - loss: 0.7516 - accuracy: 0.7741 - val_loss: 0.5780 - val_accuracy: 0.8423
Epoch 119/200
428/428 [==============================] - 1s 1ms/step - loss: 0.7098 - accuracy: 0.7910 - val_loss: 0.5590 - val_accuracy: 0.8404
Epoch 120/200
428/428 [==============================] - 1s 1ms/step - loss: 0.7420 - accuracy: 0.7761 - val_loss: 0.5758 - val_accuracy: 0.8372
Epoch 121/200
428/428 [==============================] - 1s 1ms/step - loss: 0.7582 - accuracy: 0.7713 - val_loss: 0.5790 - val_ac

428/428 [==============================] - 1s 1ms/step - loss: 0.9566 - accuracy: 0.6881 - val_loss: 0.6988 - val_accuracy: 0.7848
Epoch 28/200
428/428 [==============================] - 1s 1ms/step - loss: 0.9559 - accuracy: 0.6904 - val_loss: 0.7225 - val_accuracy: 0.7805
Epoch 29/200
428/428 [==============================] - 1s 1ms/step - loss: 0.9494 - accuracy: 0.6957 - val_loss: 0.7943 - val_accuracy: 0.7589
Epoch 30/200
428/428 [==============================] - 1s 1ms/step - loss: 0.9589 - accuracy: 0.6917 - val_loss: 0.7340 - val_accuracy: 0.7745
Epoch 31/200
428/428 [==============================] - 1s 1ms/step - loss: 0.9145 - accuracy: 0.7075 - val_loss: 0.7022 - val_accuracy: 0.7810
Epoch 32/200
428/428 [==============================] - 1s 1ms/step - loss: 0.9202 - accuracy: 0.7060 - val_loss: 0.7082 - val_accuracy: 0.7805
Epoch 33/200
428/428 [==============================] - 1s 1ms/step - loss: 0.9048 - accuracy: 0.7094 - val_loss: 0.7001 - val_accuracy: 0.7805
Epoch

Epoch 84/200
428/428 [==============================] - 1s 1ms/step - loss: 0.8538 - accuracy: 0.7231 - val_loss: 0.6714 - val_accuracy: 0.7867
Epoch 85/200
428/428 [==============================] - 1s 1ms/step - loss: 0.8459 - accuracy: 0.7305 - val_loss: 0.6579 - val_accuracy: 0.7922
Epoch 86/200
428/428 [==============================] - 1s 1ms/step - loss: 0.8725 - accuracy: 0.7201 - val_loss: 0.6825 - val_accuracy: 0.7918
Epoch 87/200
428/428 [==============================] - 1s 1ms/step - loss: 0.8507 - accuracy: 0.7282 - val_loss: 0.6612 - val_accuracy: 0.7923
Epoch 88/200
428/428 [==============================] - 1s 1ms/step - loss: 0.8396 - accuracy: 0.7339 - val_loss: 0.6494 - val_accuracy: 0.8130
Epoch 89/200
428/428 [==============================] - 1s 1ms/step - loss: 0.8388 - accuracy: 0.7321 - val_loss: 0.6279 - val_accuracy: 0.8110
Epoch 90/200
428/428 [==============================] - 1s 1ms/step - loss: 0.8485 - accuracy: 0.7258 - val_loss: 0.6654 - val_accuracy:

...

Epoch 60/200
428/428 [==============================] - 2s 4ms/step - loss: 0.7587 - accuracy: 0.7639 - val_loss: 0.6059 - val_accuracy: 0.8202
Epoch 61/200
428/428 [==============================] - 2s 4ms/step - loss: 0.7336 - accuracy: 0.7730 - val_loss: 0.5864 - val_accuracy: 0.8320
Epoch 62/200
428/428 [==============================] - 2s 4ms/step - loss: 0.7403 - accuracy: 0.7696 - val_loss: 0.6078 - val_accuracy: 0.8174
Epoch 63/200
428/428 [==============================] - 2s 4ms/step - loss: 0.7924 - accuracy: 0.7506 - val_loss: 0.6436 - val_accuracy: 0.8028
Epoch 64/200
428/428 [==============================] - 2s 4ms/step - loss: 0.7743 - accuracy: 0.7557 - val_loss: 0.6598 - val_accuracy: 0.7938
Epoch 65/200
428/428 [==============================] - 2s 4ms/step - loss: 0.7926 - accuracy: 0.7484 - val_loss: 0.6307 - val_accuracy: 0.8123
Epoch 66/200
428/428 [==============================] - 2s 4ms/step - loss: 0.7733 - accuracy: 0.7574 - val_loss: 0.6134 - val_accuracy:

Epoch 117/200
428/428 [==============================] - 2s 6ms/step - loss: 0.6478 - accuracy: 0.8038 - val_loss: 0.5369 - val_accuracy: 0.8463
Epoch 118/200
428/428 [==============================] - 2s 6ms/step - loss: 0.6380 - accuracy: 0.8073 - val_loss: 0.5382 - val_accuracy: 0.8454
Epoch 119/200
428/428 [==============================] - 2s 6ms/step - loss: 0.6588 - accuracy: 0.8003 - val_loss: 0.5440 - val_accuracy: 0.8436
Epoch 120/200
428/428 [==============================] - 2s 6ms/step - loss: 0.6650 - accuracy: 0.7987 - val_loss: 0.5375 - val_accuracy: 0.8412
Epoch 121/200
428/428 [==============================] - 2s 6ms/step - loss: 0.6587 - accuracy: 0.8021 - val_loss: 0.5353 - val_accuracy: 0.8470
Epoch 122/200
428/428 [==============================] - 3s 6ms/step - loss: 0.6432 - accuracy: 0.8070 - val_loss: 0.5314 - val_accuracy: 0.8508
Epoch 123/200
428/428 [==============================] - 3s 6ms/step - loss: 0.6298 - accuracy: 0.8117 - val_loss: 0.5244 - val_ac

428/428 [==============================] - 2s 5ms/step - loss: 0.9060 - accuracy: 0.6999 - val_loss: 0.6947 - val_accuracy: 0.8019
Epoch 30/200
428/428 [==============================] - 2s 5ms/step - loss: 0.8589 - accuracy: 0.7206 - val_loss: 0.6805 - val_accuracy: 0.7954
Epoch 31/200
428/428 [==============================] - 2s 5ms/step - loss: 0.9353 - accuracy: 0.6967 - val_loss: 0.7444 - val_accuracy: 0.7707
Epoch 32/200
428/428 [==============================] - 2s 5ms/step - loss: 0.8833 - accuracy: 0.7146 - val_loss: 0.7104 - val_accuracy: 0.7752
Epoch 33/200
428/428 [==============================] - 2s 5ms/step - loss: 0.8286 - accuracy: 0.7332 - val_loss: 0.7196 - val_accuracy: 0.7747
Epoch 34/200
428/428 [==============================] - 2s 5ms/step - loss: 0.8502 - accuracy: 0.7252 - val_loss: 0.7205 - val_accuracy: 0.7734
Epoch 35/200
428/428 [==============================] - 2s 5ms/step - loss: 0.8953 - accuracy: 0.7028 - val_loss: 0.7171 - val_accuracy: 0.7666
Epoch

Epoch 86/200
428/428 [==============================] - 2s 5ms/step - loss: 0.7808 - accuracy: 0.7534 - val_loss: 0.6182 - val_accuracy: 0.8216
Epoch 87/200
428/428 [==============================] - 2s 4ms/step - loss: 0.7786 - accuracy: 0.7573 - val_loss: 0.6712 - val_accuracy: 0.7967
Epoch 88/200
428/428 [==============================] - 2s 5ms/step - loss: 0.7893 - accuracy: 0.7514 - val_loss: 0.6443 - val_accuracy: 0.8058
Epoch 89/200
428/428 [==============================] - 2s 5ms/step - loss: 0.7568 - accuracy: 0.7644 - val_loss: 0.6488 - val_accuracy: 0.7965
Epoch 90/200
428/428 [==============================] - 2s 5ms/step - loss: 0.7375 - accuracy: 0.7714 - val_loss: 0.6353 - val_accuracy: 0.7974
Epoch 91/200
428/428 [==============================] - 2s 5ms/step - loss: 0.7459 - accuracy: 0.7657 - val_loss: 0.6495 - val_accuracy: 0.7965
Epoch 92/200
428/428 [==============================] - 2s 5ms/step - loss: 0.7583 - accuracy: 0.7617 - val_loss: 0.6176 - val_accuracy:

428/428 [==============================] - 2s 5ms/step - loss: 0.7083 - accuracy: 0.7834 - val_loss: 0.5750 - val_accuracy: 0.8474
Epoch 199/200
428/428 [==============================] - 2s 5ms/step - loss: 0.6950 - accuracy: 0.7911 - val_loss: 0.5833 - val_accuracy: 0.8453
Epoch 200/200
535/535 [==============================] - 1s 1ms/step - loss: 0.5816 - accuracy: 0.8130


## Convert results to latex table

In [56]:
MLP = []
train_loss = []
train_acc = []
test_loss = []
test_acc = []

for key in historys:
    MLP.append(key)
    train_loss.append(historys[key]['loss'][200-1])
    train_acc.append(historys[key]['accuracy'][200-1])
    test_loss.append(evaluations[key][0])
    test_acc.append(evaluations[key][1])


In [58]:
df = pd.DataFrame(dict(MLP=MLP,
                      train_loss=train_loss,
                      train_acc=train_acc,
                      test_loss=test_loss,
                      test_acc=test_acc))
print(df.to_latex(index=False)) 

\begin{tabular}{lrrrr}
\toprule
         MLP &  train\_loss &  train\_acc &  test\_loss &  test\_acc \\
\midrule
    64\_64\_64 &    0.688487 &   0.793502 &   0.546890 &  0.835479 \\
   64\_64\_128 &    0.739960 &   0.770263 &   0.556264 &  0.841329 \\
   64\_64\_256 &    0.694242 &   0.787925 &   0.553381 &  0.839633 \\
   64\_64\_512 &    0.761861 &   0.763023 &   0.560574 &  0.843904 \\
   64\_128\_64 &    0.657056 &   0.803375 &   0.543313 &  0.839164 \\
  64\_128\_128 &    0.643446 &   0.806209 &   0.535997 &  0.840686 \\
  64\_128\_256 &    0.679048 &   0.788693 &   0.588958 &  0.820676 \\
  64\_128\_512 &    0.754374 &   0.756294 &   0.605408 &  0.812719 \\
   64\_256\_64 &    0.768923 &   0.761030 &   0.568224 &  0.836532 \\
  64\_256\_128 &    0.757814 &   0.749968 &   0.556796 &  0.836532 \\
  64\_256\_256 &    0.686315 &   0.788053 &   0.533915 &  0.839340 \\
  64\_256\_512 &    0.687329 &   0.790485 &   0.543429 &  0.838638 \\
   64\_512\_64 &    0.787914 &   0.747043 &   0

In [62]:
print("train_loss", min(train_loss))
print("train_acc", max(train_acc))
print("test_loss", min(test_loss))
print("test_acc", max(test_acc))


train_loss 0.6085997223854065
train_acc 0.819720983505249
test_loss 0.527176558971405
test_acc 0.847296953201294
